<a href="https://colab.research.google.com/github/afislonge/BizRecProject/blob/dev/final_second_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# code to drive mount


from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install lightfm
!pip install gensim
!pip install textblob
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=808331 sha256=69995644122b05e491a415aa789749b65c9d2f5b673e8432c59aa0518471d0c9
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [ ]:

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType, DateType
from pyspark.sql.functions import to_date
import pandas as pd
import os
from pyspark.sql.functions import udf, col
from pyspark.sql.types import DoubleType


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, when
from pyspark.sql.types import StringType

import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Filter CSV Data") \
    .getOrCreate()

# Load the CSV file into a DataFrame
review_df = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv", header=True, inferSchema=True)
review_df.show()

In [ ]:
selected_df = review_df.select("business_id", "rev_text")

# Define the save path
save_path = '/content/drive/MyDrive/ProcessedCSV/Selected_Reviews.csv'

# Save the selected DataFrame to the specified path
selected_df.write.csv(save_path, header=True, mode='overwrite')

print(f"Selected DataFrame saved to {save_path}")


selected_df.show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, count, expr
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Step 0: Initialize Spark Session
spark = SparkSession.builder.appName("BusinessReviewsAnalysisWithTopics").getOrCreate()

# Step 1: Load Dataset
review_df = spark.read.csv("/content/drive/MyDrive/ProcessedCSV/Recommender_System_Newdata.csv", header=True, inferSchema=True)

# Step 2: Preprocess the DataFrame
selected_df = review_df.select("business_id", "rev_text")

# Add sentiment labels
df = selected_df.withColumn(
    "sentiment",
    when(col("rev_text").rlike("(?i)good|great|excellent|awesome|perfect|fantastic"), "positive").otherwise("negative")
)

# Define a dictionary mapping keywords to topics
keywords_to_topics = {
    "service": "customer_service",
    "staff": "customer_service",
    "wait": "customer_service",
    "food": "food_quality",
    "taste": "food_quality",
    "flavor": "food_quality",
    "hygiene": "cleanliness",
    "clean": "cleanliness",
    "price": "pricing",
    "cost": "pricing"
}

# Create a case-when expression for topics
topic_expr = " ".join(
    [f"WHEN LOWER(rev_text) LIKE '%{keyword}%' THEN '{topic}'" for keyword, topic in keywords_to_topics.items()]
)

# Add topic column
df = df.withColumn("topic", expr(f"CASE {topic_expr} ELSE 'other' END"))

# Step 3: Balance Classes
# Count positive and negative sentiments
df.groupBy("sentiment").count().show()

# Downsample or upsample classes if imbalanced
positive_df = df.filter(col("sentiment") == "positive")
negative_df = df.filter(col("sentiment") == "negative")

# Ensure balanced data
balanced_df = positive_df.sample(False, 0.5, seed=42).unionAll(negative_df)

# Step 4: Feature Extraction Pipeline
tokenizer = Tokenizer(inputCol="rev_text", outputCol="tokens")
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
hashing_tf = HashingTF(inputCol="filtered_tokens", outputCol="rawFeatures", numFeatures=5000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
sentiment_indexer = StringIndexer(inputCol="sentiment", outputCol="sentimentIndex")

# Step 5: Model Definition
rf = RandomForestClassifier(
    labelCol="sentimentIndex",
    featuresCol="features",
    numTrees=20,
    maxDepth=5,
    seed=42
)

# Build the Pipeline
pipeline = Pipeline(stages=[tokenizer, remover, hashing_tf, idf, sentiment_indexer, rf])

# Step 6: Split Data
train_df, test_df = balanced_df.randomSplit([0.8, 0.2], seed=42)

# Step 7: Train the Model
model = pipeline.fit(train_df)

# Step 8: Test the Model
predictions = model.transform(test_df)

# Rename the prediction column for clarity (optional)
predictions = predictions.withColumnRenamed("prediction", "sentimentPrediction")

# Evaluate Model
evaluator = MulticlassClassificationEvaluator(
    labelCol="sentimentIndex",
    predictionCol="sentimentPrediction",
    metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy:.2f}")

# Debug Predictions
predictions.select("business_id", "rev_text", "sentiment", "sentimentPrediction", "topic").show(truncate=False)

# Step 9: Calculate Sentiment Percentages for Each Business
sentiment_count = predictions.groupBy("business_id", "sentimentPrediction").agg(count("sentimentPrediction").alias("sentiment_count"))
total_reviews = predictions.groupBy("business_id").agg(count("rev_text").alias("total_reviews"))

sentiment_percentage = sentiment_count.join(total_reviews, on="business_id")
sentiment_percentage = sentiment_percentage.withColumn(
    "sentiment_percentage",
    (col("sentiment_count") / col("total_reviews") * 100)
)

# Step 10: Show Topic Distribution for Each Business
topic_count = predictions.groupBy("business_id", "topic").agg(count("topic").alias("topic_count"))
total_reviews = predictions.groupBy("business_id").agg(count("rev_text").alias("total_reviews"))

topic_percentage = topic_count.join(total_reviews, on="business_id")
topic_percentage = topic_percentage.withColumn(
    "topic_percentage",
    (col("topic_count") / col("total_reviews") * 100)
)

# Show Final Results
print("Sentiment Percentages:")
sentiment_percentage.show(truncate=False)

print("Topic Percentages:")
topic_percentage.show(truncate=False)


+---------+-------+
|sentiment|  count|
+---------+-------+
| positive|2470506|
| negative|1227378|
+---------+-------+

Test Accuracy: 0.84
+----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# # Save the PySpark model
# model_path = "/content/drive/MyDrive/topic_senti_model"
# model.write().overwrite().save(model_path)
# print(f"Model saved at {model_path}")


Model saved at /content/drive/MyDrive/topic_senti_model


In [ ]:
# from pyspark.ml import PipelineModel

# # Load the saved model
# loaded_model = PipelineModel.load("/content/drive/MyDrive/topic_senti_model")
# print("Model loaded successfully!")


Model loaded successfully!
